# Machine Learning Coarse Graining

## Overview:

This tutorial demonstrates how to use a pre-trained machine learning model to predict forces in a coarse-grained system. We will use the flowerMD package to load a pre-trained pytorch model and apply it to system of spherical (isotropic) particles.
This model uses neighbor lists to predict forces between particles in the system. The model is trained on a dataset of Lennard-Jones systems with 200 particles.

We use a pre-trained pytorch neural network to determine the forces between particles based on the relative distance vector between each particle and its 40th nearest neighbors. This model was trained for 1000 epochs with a batch size of 128 and an Adam optimizer. The best model checkpoint is saved to a file, which we will load and apply to a simulation.

The `load_isotropic_custom_force` function loads the pre-trained model and creates a custom force object that can be applied to a simulation. This custom force object is like a look-up function for the simulation, providing the net force on each particle at each time step.

In [1]:
from flowermd.modules.coarse_graining_ML.isotropic_coarse_graining import (
    load_isotropic_custom_force,
    LJNeighborModelConfig,
)
from flowermd import Simulation

The `load_isotropic_custom_force` function loads the pre-trained model and creates a custom force object that can be applied to a simulation. This custom force object is like a look-up function for the simulation, providing the net force on each particle at each time step.

In [2]:
best_model_path = "data/best_model_checkpoint.pth"
lj_system = "data/LJ_system_N200.gsd"
model_config = LJNeighborModelConfig(
    number_neighbors=40,
    hidden_dim=128,
    n_layers=3,
    act_fn="Tanh",
    dropout=0.3,
    batch_norm=False,
    box_len=6,
    prior_energy=True,
    prior_energy_sigma=1,
    prior_energy_n=12,
)
custom_force = load_isotropic_custom_force(
    model_config, best_model_path=best_model_path
)
sim = Simulation(
    initial_state=lj_system,
    forcefield=[custom_force],
    gsd_file_name="LJ_trajectory.gsd",
    log_file_name="LJ_log.txt",
)
sim.run_NVT(temperature=3.0, duration=10000, tau_kt=0.1)
sim.flush_writers()
print("done")

/Users/marjan/Documents/code-base/forks/marjan/flowerMD/flowermd/modules/coarse_graining_ML/isotropic_coarse_graining.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m

Initializing simulation state from a GSD file.
Step 4499 of 10000; TPS: 68.17; ETA: 1.3 minutes
Step 9999 of 10000; TPS: 68.67; ETA: 0.0 minutes
done
